IMF에서 제공하는 국가별 GDP를 구하세요. (https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29)

국가별 GDP를 확인할 수 있는 테이블을 만드세요.

해당 테이블에는 GDP가 높은 국가들이 먼저 나와야 합니다.

GDP의 단위는 1B USD이어야 하고 소수점 2자리까지만 표시해 주세요.

IMF에서 매년 2회 이 자료를 제공하기 때문에 정보가 갱신되더라도 해당 코드를 재사용해서 정보를 얻을 수 있어야 합니다.



GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력해야 합니다.

각 Region별로 top5 국가의 GDP 평균을 구해서 화면에 출력해야 합니다.

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import requests

# Extract

# url에서 원본 HTML 로드, bs4 객체회
print("Start Extraction")
response = requests.get('https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29')
if response.status_code != 200:
    print(response.status_code)
    raise Exception("Failed to load page")
else:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

# 
table = soup.find('table', {'class':'wikitable'})
rows = table.find_all('tr')

print("End Extraction")


Start Extraction
End Extraction


In [2]:
print("Start Transformation")

countrys = []
gdps = []

for i, row in enumerate(rows):
    # 테이블 메타데이터
    if i in [0, 1, 2]:
        continue
    # 각 행의 열 정보들 리스트화
    cols = row.find_all('td')
    for j, col in enumerate(cols):
        # 국가 정보
        if j == 0:
            # print(col.get_text(strip=True))
            countrys.append(col.get_text(strip=True))
        # IMF의 GDP정보.
        if j == 1:
            content = col.get_text(strip=True)
            # IMF 미가입 국가
            if content == '—':
                gdps.append(-1)
            else:
                gdps.append(float(content.replace(',', '')))

Start Transformation


In [3]:
# Pandas DataFrame화
df = pd.DataFrame({'Country':countrys, 'GDP':gdps})

# IMF 미가입 국가의 데이터값 Not a Number로 변경
df['GDP'] = df['GDP'].replace(-1, np.nan)
# GDP 값 1B USD 단위로 변경
df['GDP'] = (df['GDP']/1000).round(2)

# 생성한 DF .json으로 저장
# df.to_json("Countries_by_GDP.json")

In [4]:
# GDP 100B USD 이상의 국가 출력
df_100B = df[df['GDP'] >= 100]
df_100B

,Country,GDP
0,United States,30507.22
1,China,19231.71
2,Germany,4744.80
3,India,4187.02
4,Japan,4186.43
...,...,...
69,Bulgaria,117.01
70,Angola,113.34
71,Venezuela,108.51
72,Oman,104.35
